In [1]:
import numpy as np
import tensorflow as tf
import keras
from matplotlib import pyplot as plt
from keras.models import Model
from keras.layers import LSTM, Dense, Input, Dropout

Using TensorFlow backend.


In [2]:
lines = []
with open("ph.txt") as phfile:
    lines = phfile.readlines()

lines = [k.split('\t') for k in lines]
lines = [[k[0], k[1].split(' ')] for  k in lines]
lines = [[k[0], k[1][:-1]+[k[1][-1][:-1]]] for k in lines]

lines[:5]

[['aback', ['AH', 'B', 'AE', 'K']],
 ['abandon', ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N']],
 ['abandoned', ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'D']],
 ['abandoning', ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'IH', 'NG']],
 ['abandons', ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'Z']]]

In [3]:
xraw = [k[0] for k in lines]
yraw = [k[1] for k in lines]

In [4]:
yraw = [['\s'] + k + ['\e'] for k in yraw]

In [5]:
phoneme_array = [
    '\s',    'AA',    'AE',    'AH',    'AO',
    'AW',    'AY',    'B',    'CH',    'D',
    'DH',    'EH',    'ER',    'EY',    'F',
    'G',    'HH',    'IH',    'IY',    'JH',
    'K',    'L',    'M',    'N',    'NG',
    'OW',    'OY',    'P',    'R',    'S',
    'SH',    'T',    'TH',    'UH',    'UW',
    'V',    'W',    'Y',    'Z',    'ZH',  '\e'
]

In [6]:
phonemes_count = len(phoneme_array)
chars_count = 26
phonemes_count, chars_count

(41, 26)

In [7]:
phoneme_2_index = dict(zip(phoneme_array, range(len(phoneme_array))))
index_2_phoneme = dict(zip(range(len(phoneme_array)), phoneme_array))

In [8]:
char_2_index = dict(zip([chr(ord('a')+k) for k in range(26)], range(26) ))
index_2_char = dict(zip(range(26), [chr(ord('a')+k) for k in range(26)] ))

In [9]:
max_word_len = max([len(k) for k in xraw])
max_word_len

20

In [10]:
max_phonemes_len = max([len(k) for k in yraw])
max_phonemes_len

18

In [11]:
# no_entries chr_idx vct_idx
# 0-len(xraw)  0-19  0-25

xdata = np.zeros(shape=(len(xraw), max_word_len, 26), dtype='float32')

In [12]:
for i in range(len(xraw)):
    word = xraw[i]
    for j in range(len(word)):
        xdata[i,j,char_2_index[word[j]]] = 1

In [13]:
# no_entries ph_idx vct_idx
# 0-len(xraw)  0-max_wordlen  0-25
# ---> We are padding phonemes with start and stop symbols

ydata = np.zeros(shape=(len(yraw), max_phonemes_len, phonemes_count), dtype='float32')

In [14]:
for i in range(len(yraw)):
    phword = yraw[i]
    print(phword)
    for j in range(len(phword)):
        ydata[i,j,phoneme_2_index[phword[j]]] = 1

['\\s', 'AH', 'B', 'AE', 'K', '\\e']
['\\s', 'AH', 'B', 'AE', 'N', 'D', 'AH', 'N', '\\e']
['\\s', 'AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'D', '\\e']
['\\s', 'AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'IH', 'NG', '\\e']
['\\s', 'AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'Z', '\\e']
['\\s', 'AH', 'B', 'AE', 'SH', '\\e']
['\\s', 'AH', 'B', 'AE', 'SH', 'T', '\\e']
['\\s', 'AH', 'B', 'EY', 'T', '\\e']
['\\s', 'AH', 'B', 'EY', 'T', 'IH', 'D', '\\e']
['\\s', 'AH', 'B', 'EY', 'T', 'S', '\\e']
['\\s', 'AH', 'B', 'EY', 'T', 'IH', 'NG', '\\e']
['\\s', 'AH', 'B', 'R', 'IY', 'V', 'IY', 'EY', 'T', '\\e']
['\\s', 'AH', 'B', 'R', 'IY', 'V', 'IY', 'EY', 'T', 'S', '\\e']
['\\s', 'AH', 'B', 'R', 'IY', 'V', 'IY', 'EY', 'T', 'IH', 'NG', '\\e']
['\\s', 'AE', 'B', 'D', 'AH', 'K', 'EY', 'T', '\\e']
['\\s', 'AE', 'B', 'D', 'AH', 'K', 'EY', 'T', 'AH', 'D', '\\e']
['\\s', 'AE', 'B', 'D', 'AH', 'K', 'EY', 'T', 'S', '\\e']
['\\s', 'AE', 'B', 'D', 'IH', 'K', 'EY', 'T', 'IH', 'NG', '\\e']
['\\s', 'AE', 'B', 'D', 'AH', 'K', 'T'

['\\s', 'B', 'EY', 'K', 'S', '\\e']
['\\s', 'B', 'EY', 'K', 'IH', 'NG', '\\e']
['\\s', 'B', 'AE', 'L', 'AH', 'N', 'S', '\\e']
['\\s', 'B', 'AE', 'L', 'AH', 'N', 'S', 'T', '\\e']
['\\s', 'B', 'AE', 'L', 'AH', 'N', 'S', 'IH', 'NG', '\\e']
['\\s', 'B', 'AO', 'L', 'D', '\\e']
['\\s', 'B', 'AO', 'L', 'D', 'ER', '\\e']
['\\s', 'B', 'AO', 'L', 'D', 'IH', 'NG', '\\e']
['\\s', 'B', 'AO', 'L', 'D', 'IH', 'NG', 'ER', '\\e']
['\\s', 'B', 'AO', 'L', 'D', 'L', 'IY', '\\e']
['\\s', 'B', 'EY', 'L', '\\e']
['\\s', 'B', 'EY', 'L', 'F', 'AH', 'L', '\\e']
['\\s', 'B', 'EY', 'L', 'Z', '\\e']
['\\s', 'B', 'EY', 'L', 'IH', 'NG', '\\e']
['\\s', 'B', 'AO', 'L', 'K', 'AH', 'N', 'AY', 'Z', '\\e']
['\\s', 'B', 'AO', 'L', 'K', 'AH', 'N', 'AY', 'Z', 'D', '\\e']
['\\s', 'B', 'AO', 'K', 'T', '\\e']
['\\s', 'B', 'AO', 'K', 'IH', 'NG', '\\e']
['\\s', 'B', 'AO', 'K', 'S', '\\e']
['\\s', 'B', 'AO', 'K', 'IY', '\\e']
['\\s', 'B', 'AO', 'L', '\\e']
['\\s', 'B', 'AE', 'L', 'AH', 'S', 'T', '\\e']
['\\s', 'B', 'AO', 'L', 'D',

['\\s', 'B', 'R', 'EY', 'N', 'W', 'AA', 'SH', 'T', '\\e']
['\\s', 'B', 'R', 'EY', 'N', 'W', 'AA', 'SH', 'AH', 'Z', '\\e']
['\\s', 'B', 'R', 'EY', 'N', 'W', 'AA', 'SH', 'IH', 'NG', '\\e']
['\\s', 'B', 'R', 'EY', 'N', 'IY', '\\e']
['\\s', 'B', 'R', 'EY', 'Z', '\\e']
['\\s', 'B', 'R', 'EY', 'Z', 'D', '\\e']
['\\s', 'B', 'R', 'EY', 'K', '\\e']
['\\s', 'B', 'R', 'EY', 'K', 'T', '\\e']
['\\s', 'B', 'R', 'EY', 'K', 'S', '\\e']
['\\s', 'B', 'R', 'EY', 'K', 'IH', 'NG', '\\e']
['\\s', 'B', 'R', 'AE', 'N', 'CH', '\\e']
['\\s', 'B', 'R', 'AE', 'N', 'CH', 'T', '\\e']
['\\s', 'B', 'R', 'AE', 'N', 'CH', 'IH', 'NG', '\\e']
['\\s', 'B', 'R', 'AE', 'N', 'D', '\\e']
['\\s', 'B', 'R', 'AE', 'N', 'D', 'IH', 'D', '\\e']
['\\s', 'B', 'R', 'AE', 'N', 'D', 'Z', '\\e']
['\\s', 'B', 'R', 'AE', 'N', 'D', 'IH', 'NG', '\\e']
['\\s', 'B', 'R', 'AE', 'N', 'D', 'IH', 'SH', '\\e']
['\\s', 'B', 'R', 'AE', 'N', 'D', 'IH', 'SH', 'T', '\\e']
['\\s', 'B', 'R', 'AE', 'N', 'D', 'IH', 'SH', 'IH', 'Z', '\\e']
['\\s', 'B', 'R', 

['\\s', 'K', 'L', 'AE', 'M', 'ER', 'IH', 'NG', '\\e']
['\\s', 'K', 'L', 'AE', 'M', 'P', '\\e']
['\\s', 'K', 'L', 'AE', 'M', 'P', 'T', '\\e']
['\\s', 'K', 'L', 'AE', 'M', 'P', 'IH', 'NG', '\\e']
['\\s', 'K', 'L', 'AE', 'M', 'P', 'S', '\\e']
['\\s', 'K', 'L', 'AE', 'M', 'Z', '\\e']
['\\s', 'K', 'L', 'AE', 'N', 'D', 'EH', 'S', 'T', 'IH', 'N', '\\e']
['\\s', 'K', 'L', 'AE', 'NG', '\\e']
['\\s', 'K', 'L', 'AE', 'NG', 'IH', 'NG', '\\e']
['\\s', 'K', 'L', 'AE', 'NG', 'K', '\\e']
['\\s', 'K', 'L', 'AE', 'NG', 'K', 'IH', 'NG', '\\e']
['\\s', 'K', 'L', 'AE', 'N', 'IH', 'SH', '\\e']
['\\s', 'K', 'L', 'AE', 'P', '\\e']
['\\s', 'K', 'L', 'AE', 'P', 'B', 'AO', 'R', 'D', '\\e']
['\\s', 'K', 'L', 'AE', 'P', 'B', 'AO', 'R', 'D', 'Z', '\\e']
['\\s', 'K', 'L', 'AE', 'P', 'T', '\\e']
['\\s', 'K', 'L', 'AE', 'P', 'IH', 'NG', '\\e']
['\\s', 'K', 'L', 'AE', 'P', 'S', '\\e']
['\\s', 'K', 'L', 'EH', 'R', 'AH', 'F', 'AY', 'D', '\\e']
['\\s', 'K', 'L', 'EH', 'R', 'AH', 'F', 'AY', 'Z', '\\e']
['\\s', 'K', 'L', 'E

['\\s', 'K', 'R', 'AW', 'CH', 'T', '\\e']
['\\s', 'K', 'R', 'AW', 'CH', 'IH', 'NG', '\\e']
['\\s', 'K', 'R', 'OW', '\\e']
['\\s', 'K', 'R', 'AW', 'D', '\\e']
['\\s', 'K', 'R', 'AW', 'D', 'IH', 'NG', '\\e']
['\\s', 'K', 'R', 'AW', 'D', 'Z', '\\e']
['\\s', 'K', 'R', 'OW', 'D', '\\e']
['\\s', 'K', 'R', 'OW', 'IH', 'NG', '\\e']
['\\s', 'K', 'R', 'AW', 'N', '\\e']
['\\s', 'K', 'R', 'AW', 'N', 'D', '\\e']
['\\s', 'K', 'R', 'AW', 'N', 'IH', 'NG', '\\e']
['\\s', 'K', 'R', 'AW', 'N', 'Z', '\\e']
['\\s', 'K', 'R', 'OW', 'Z', '\\e']
['\\s', 'K', 'R', 'UW', 'SH', 'AH', 'L', '\\e']
['\\s', 'K', 'R', 'UW', 'S', 'AH', 'F', 'AY', 'D', '\\e']
['\\s', 'K', 'R', 'UW', 'S', 'AH', 'F', 'AY', '\\e']
['\\s', 'K', 'R', 'AH', 'D', 'IY', '\\e']
['\\s', 'K', 'R', 'UW', 'D', '\\e']
['\\s', 'K', 'R', 'UW', 'D', 'L', 'IY', '\\e']
['\\s', 'K', 'R', 'UW', 'L', 'AH', 'S', 'T', '\\e']
['\\s', 'K', 'R', 'UW', 'L', 'IY', '\\e']
['\\s', 'K', 'R', 'UW', 'Z', '\\e']
['\\s', 'K', 'R', 'UW', 'Z', 'D', '\\e']
['\\s', 'K', 'R',

...

['\\s', 'R', 'IH', 'P', 'S', '\\e']
['\\s', 'R', 'AY', 'Z', '\\e']
['\\s', 'R', 'IH', 'Z', 'AH', 'N', '\\e']
['\\s', 'R', 'AY', 'Z', 'IH', 'NG', '\\e']
['\\s', 'R', 'AY', 'Z', 'IH', 'NG', 'ER', '\\e']
['\\s', 'R', 'IH', 'S', 'K', '\\e']
['\\s', 'R', 'IH', 'S', 'K', 'T', '\\e']
['\\s', 'R', 'IH', 'S', 'K', 'IY', 'ER', '\\e']
['\\s', 'R', 'IH', 'S', 'K', 'IY', 'AH', 'S', 'T', '\\e']
['\\s', 'R', 'IH', 'S', 'K', 'IH', 'NG', '\\e']
['\\s', 'R', 'IH', 'S', 'K', 'L', 'AH', 'S', '\\e']
['\\s', 'R', 'IH', 'S', 'K', 'S', '\\e']
['\\s', 'R', 'IH', 'S', 'K', 'IY', '\\e']
['\\s', 'R', 'IH', 'S', 'K', 'EY', '\\e']
['\\s', 'R', 'IH', 'CH', 'UW', 'AH', 'L', '\\e']
['\\s', 'R', 'IH', 'CH', 'UW', 'AH', 'L', 'IH', 'S', 'T', 'IH', 'K', '\\e']
['\\s', 'R', 'IH', 'T', 'S', 'IY', 'AH', 'S', 'T', '\\e']
['\\s', 'R', 'IH', 'T', 'S', 'IY', '\\e']
['\\s', 'R', 'AY', 'V', 'AH', 'L', '\\e']
['\\s', 'R', 'AY', 'V', 'AH', 'L', 'D', '\\e']
['\\s', 'R', 'AY', 'V', 'AH', 'L', 'Z', '\\e']
['\\s', 'R', 'IH', 'V', 'AH', 

['\\s', 'S', 'L', 'IY', 'P', '\\e']
['\\s', 'S', 'L', 'IY', 'P', 'AH', 'L', 'IY', '\\e']
['\\s', 'S', 'L', 'IY', 'P', 'IH', 'NG', '\\e']
['\\s', 'S', 'L', 'IY', 'P', 'L', 'AH', 'S', '\\e']
['\\s', 'S', 'L', 'IY', 'P', 'S', '\\e']
['\\s', 'S', 'L', 'IY', 'P', 'W', 'AO', 'K', '\\e']
['\\s', 'S', 'L', 'IY', 'P', 'W', 'AO', 'K', 'IH', 'NG', '\\e']
['\\s', 'S', 'L', 'IY', 'P', 'IY', '\\e']
['\\s', 'S', 'L', 'IY', 'T', '\\e']
['\\s', 'S', 'L', 'IY', 'V', 'D', '\\e']
['\\s', 'S', 'L', 'IY', 'V', 'L', 'IH', 'S', '\\e']
['\\s', 'S', 'L', 'EY', '\\e']
['\\s', 'S', 'L', 'EH', 'N', 'D', 'ER', '\\e']
['\\s', 'S', 'L', 'EH', 'P', 'T', '\\e']
['\\s', 'S', 'L', 'UW', 'TH', '\\e']
['\\s', 'S', 'L', 'UW', 'TH', 'IH', 'NG', '\\e']
['\\s', 'S', 'L', 'UW', 'TH', 'S', '\\e']
['\\s', 'S', 'L', 'UW', '\\e']
['\\s', 'S', 'L', 'AY', 'S', '\\e']
['\\s', 'S', 'L', 'AY', 'S', 'T', '\\e']
['\\s', 'S', 'L', 'AY', 'S', 'IH', 'NG', '\\e']
['\\s', 'S', 'L', 'IH', 'K', '\\e']
['\\s', 'S', 'L', 'IH', 'K', 'T', '\\e']
['\

['\\s', 'S', 'AH', 'L', 'F', 'ER', '\\e']
['\\s', 'S', 'AH', 'L', 'F', 'Y', 'UH', 'R', 'IH', 'K', '\\e']
['\\s', 'S', 'AH', 'L', 'K', '\\e']
['\\s', 'S', 'AH', 'L', 'K', 'T', '\\e']
['\\s', 'S', 'AH', 'L', 'K', 'IH', 'NG', '\\e']
['\\s', 'S', 'AH', 'L', 'AH', 'N', '\\e']
['\\s', 'S', 'AH', 'L', 'IY', 'D', '\\e']
['\\s', 'S', 'AH', 'L', 'IY', '\\e']
['\\s', 'S', 'AH', 'L', 'F', 'ER', '\\e']
['\\s', 'S', 'AH', 'L', 'T', 'R', 'IY', '\\e']
['\\s', 'S', 'AH', 'M', '\\e']
['\\s', 'S', 'UW', 'M', 'AA', 'T', 'R', 'AH', 'N', '\\e']
['\\s', 'S', 'AH', 'M', 'EH', 'R', 'IH', 'L', 'IY', '\\e']
['\\s', 'S', 'AH', 'M', 'ER', 'AY', 'Z', '\\e']
['\\s', 'S', 'AH', 'M', 'ER', 'AY', 'Z', 'D', '\\e']
['\\s', 'S', 'AH', 'M', 'ER', 'AY', 'Z', 'IH', 'Z', '\\e']
['\\s', 'S', 'AH', 'M', 'ER', 'AY', 'Z', 'IH', 'NG', '\\e']
['\\s', 'S', 'AH', 'M', 'ER', 'IY', '\\e']
['\\s', 'S', 'AH', 'M', 'D', '\\e']
['\\s', 'S', 'AH', 'M', 'ER', '\\e']
['\\s', 'S', 'AH', 'M', 'ER', 'Z', '\\e']
['\\s', 'S', 'AH', 'M', 'IH', 'NG'

['\\s', 'AH', 'N', 'AO', 'L', 'T', 'ER', 'D', '\\e']
['\\s', 'AH', 'N', 'AE', 'M', 'B', 'IH', 'G', 'Y', 'AH', 'W', 'AH', 'S', '\\e']
['\\s', 'AH', 'N', 'AE', 'M', 'B', 'IH', 'G', 'Y', 'UW', 'AH', 'S', 'L', 'IY', '\\e']
['\\s', 'AH', 'N', 'AH', 'M', 'EH', 'N', 'D', 'IH', 'D', '\\e']
['\\s', 'Y', 'UW', 'N', 'AE', 'N', 'AH', 'M', 'AH', 'S', '\\e']
['\\s', 'Y', 'UW', 'N', 'AE', 'N', 'AH', 'M', 'AH', 'S', 'L', 'IY', '\\e']
['\\s', 'AH', 'N', 'AH', 'N', 'AW', 'N', 'S', 'T', '\\e']
['\\s', 'AH', 'N', 'AE', 'N', 'S', 'ER', 'AH', 'B', 'AH', 'L', '\\e']
['\\s', 'AH', 'N', 'AE', 'N', 'S', 'ER', 'D', '\\e']
['\\s', 'AH', 'N', 'AE', 'N', 'T', 'IH', 'S', 'AH', 'P', 'EY', 'T', 'IH', 'D', '\\e']
['\\s', 'AH', 'N', 'AH', 'P', 'AA', 'L', 'AH', 'JH', 'EH', 'T', 'IH', 'K', '\\e']
['\\s', 'AH', 'N', 'AH', 'P', 'IY', 'L', 'IH', 'NG', '\\e']
['\\s', 'AH', 'N', 'AE', 'P', 'AH', 'T', 'AY', 'Z', 'IH', 'NG', '\\e']
['\\s', 'AH', 'N', 'AH', 'P', 'R', 'IY', 'SH', 'IY', 'EY', 'T', 'IH', 'D', '\\e']
['\\s', 'AH', 'N

['\\s', 'W', 'IH', 'CH', '\\e']
['\\s', 'W', 'IH', 'CH', 'IH', 'NG', '\\e']
['\\s', 'W', 'IH', 'DH', 'ER', '\\e']
['\\s', 'W', 'IH', 'DH', 'ER', 'D', '\\e']
['\\s', 'W', 'IH', 'DH', 'ER', 'IH', 'NG', '\\e']
['\\s', 'W', 'IH', 'DH', 'ER', 'Z', '\\e']
['\\s', 'W', 'IH', 'TH', 'HH', 'EH', 'L', 'D', '\\e']
['\\s', 'W', 'IH', 'TH', 'HH', 'OW', 'L', 'D', '\\e']
['\\s', 'W', 'IH', 'TH', 'HH', 'OW', 'L', 'D', 'IH', 'NG', '\\e']
['\\s', 'W', 'IH', 'TH', 'HH', 'OW', 'L', 'D', 'Z', '\\e']
['\\s', 'W', 'IH', 'TH', 'S', 'T', 'AE', 'N', 'D', '\\e']
['\\s', 'W', 'IH', 'TH', 'S', 'T', 'AE', 'N', 'D', 'IH', 'NG', '\\e']
['\\s', 'W', 'IH', 'TH', 'S', 'T', 'AE', 'N', 'D', 'Z', '\\e']
['\\s', 'W', 'IH', 'TH', 'S', 'T', 'UH', 'D', '\\e']
['\\s', 'W', 'IH', 'T', 'L', 'AH', 'S', '\\e']
['\\s', 'W', 'IH', 'T', 'N', 'AH', 'S', '\\e']
['\\s', 'W', 'IH', 'T', 'N', 'AH', 'S', 'T', '\\e']
['\\s', 'W', 'IH', 'T', 'N', 'AH', 'S', 'IH', 'NG', '\\e']
['\\s', 'W', 'IH', 'T', 'AH', 'L', 'IY', '\\e']
['\\s', 'W', 'IH', '

---------------------------

In [15]:
encoder_lstm_output_size = 64
decoder_lstm_output_size = 64

In [16]:
# The first lstm feeds on word vec
encoder_lstm_input = Input(
    shape=(None, chars_count),
    dtype='float32',
    name='encoder_lstm_input'
)

In [17]:
# This lstm should return the internal state C-memory H-hypothesis
encoder_lstm = LSTM(
    encoder_lstm_output_size,
    return_state=True,
    return_sequences=True
)

In [18]:
encoder_lstm_output, encoder_lstm_h, encoder_lstm_c = encoder_lstm(encoder_lstm_input)

In [19]:
# This bad boy will be the initial state for decoder lstm
encoder_lstm_state = [encoder_lstm_h, encoder_lstm_c]

------------

In [20]:
# We pass phonemes that we expect in the same order
# - The idea is that, based on internal state from encoder and start symbol - the first phoneme will be generated
# - This first generated phoneme can be fed along with new state of decoder as next iteration input.

# During training, since we already know whats what - we can feed input as known phonemes and output shall be
# one timestep left shifted. i.e., input -> \s, <first_phoneme>... | output -> <first_phoneme>, <second_phoneme>...

decoder_lstm_input = Input(
    shape=(None, phonemes_count),
    dtype='float32',
    name='decoder_lstm_input'
)

In [21]:
# Decoder's output and state is needed - we'll use decoder's output state in first iteration as input for next
decoder_lstm = LSTM(
    decoder_lstm_output_size,
    return_state=True,
    return_sequences=True
)

In [22]:
decoder_lstm_inputs, decoder_lstm_h, decoder_lstm_c = decoder_lstm(
    decoder_lstm_input,
    initial_state=encoder_lstm_state)

In [23]:
decoder_dense_layer = Dense(phonemes_count, activation='softmax')
decoder_dense_output = decoder_dense_layer(decoder_lstm_inputs)

------------

In [24]:
enc_dec_model = Model(
    inputs=[encoder_lstm_input, decoder_lstm_input], # <word representation>, <expected phonemes representation>
    outputs=decoder_dense_output # <one timestep left shifted phonemes>
)

In [25]:
enc_dec_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [26]:
enc_dec_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_lstm_input (InputLayer) (None, None, 26)     0                                            
__________________________________________________________________________________________________
decoder_lstm_input (InputLayer) (None, None, 41)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 64), ( 23296       encoder_lstm_input[0][0]         
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 64), ( 27136       decoder_lstm_input[0][0]         
                                                                 lstm_1[0][1]                     
          

In [27]:
_shift_ydata = np.zeros(shape=ydata.shape, dtype='float32')
_shift_ydata[:,:,:-1] = ydata[:,:,1:] 

even_indices = [2*k for k in range(int(xdata.shape[0]/2))]
odd_indices = [2*k+1 for k in range(int(xdata.shape[0]/2) - 1)]

_train_xdata = xdata[even_indices]
_train_ydata = ydata[even_indices]
_train_shift_ydata = _shift_ydata[even_indices]

_test_xdata = xdata[odd_indices]
_test_ydata = ydata[odd_indices]
_test_shift_ydata = _shift_ydata[odd_indices]

In [28]:
enc_dec_train_history = enc_dec_model.fit(
    x=[_train_xdata, _train_ydata],
    y=_train_shift_ydata,
    batch_size=64,
    epochs=40,
    verbose=1,
    validation_split=0.1
)

Train on 10874 samples, validate on 1209 samples
Epoch 1/40
10874/10874 [==============================] - 20s 2ms/step - loss: 1.0889 - val_loss: 0.8250
Epoch 2/40
10874/10874 [==============================] - 13s 1ms/step - loss: 0.3967 - val_loss: 0.1521
Epoch 3/40
10874/10874 [==============================] - 13s 1ms/step - loss: 0.0374 - val_loss: 0.0088
Epoch 4/40
10874/10874 [==============================] - 14s 1ms/step - loss: 0.0021 - val_loss: 8.3192e-04
Epoch 5/40
10874/10874 [==============================] - 13s 1ms/step - loss: 1.6830e-04 - val_loss: 8.2388e-05
Epoch 6/40
10874/10874 [==============================] - 12s 1ms/step - loss: 2.0536e-05 - val_loss: 1.2630e-05
Epoch 7/40
10874/10874 [==============================] - 16s 1ms/step - loss: 4.2821e-06 - val_loss: 3.5715e-06
Epoch 8/40
10874/10874 [==============================] - 14s 1ms/step - loss: 1.4783e-06 - val_loss: 1.8148e-06
Epoch 9/40
  192/10874 [..............................] - ETA: 12s - loss: 

KeyboardInterrupt: 

----

In [29]:
'''
Now, we reassemble the model using trained variables.

encoder is a model that takes lstm1_inp and generates internal_states
'''
encoder_model = Model(
    inputs=encoder_lstm_input,
    outputs=encoder_lstm_state
)

In [30]:
'''
Decoder has 2 inputs - the hypotheses/state and classification/output of encoder
Together, they make up decoder input.
'''
decoder_input_h = Input(shape=(decoder_lstm_output_size,), name='decoder_input_h')
decoder_input_c = Input(shape=(decoder_lstm_output_size,), name='decoder_input_c')

decoder_initial_state = [decoder_input_h, decoder_input_c]

# This lstm constructs internal rep from lstm2_inputs = phonemes vector
# and internal state of first lstm
decoder_dense_inputs, decoder_output_h, decoder_output_c = decoder_lstm(
    decoder_lstm_input,
    initial_state=decoder_initial_state
)

decoder_input_state = [decoder_output_h, decoder_output_c] # internal rep

In [31]:
# This internal rep is passed through a Dense NN(the one we previously trained)
decoder_dense_outputs = decoder_dense_layer(decoder_dense_inputs)

In [32]:
# Input to the deocder model are
# > lstm2_inputs = previous phonemes
# > H state of previous LSTM(1)
# > C state of previous LSTM(1)

# Outputs of this model are
# > Decoder dense layer's output
# > H state of this LSTM(2)
# > C state of this LSTM(2)
decoder_model = Model(
    inputs = [decoder_lstm_input] + decoder_initial_state,
    outputs = [decoder_dense_outputs] + decoder_input_state)

In [33]:
# Reverse-lookup tables
reverse_input_char_index = index_2_char
reverse_target_char_index = index_2_phoneme

In [34]:
def decode_sequence(input_batch):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_batch)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, phonemes_count))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, phoneme_2_index['\s']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = []
    i = 0
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += [sampled_char]

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\e' or
           len(decoded_sentence) > max_phonemes_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, phonemes_count))
        target_seq[0, 0, sampled_token_index] = 1.
        i+=1

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_batch = _test_xdata[seq_index]
    decoded_sentence = decode_sequence(np.reshape(input_batch, newshape=(1, 20, 26)))
    print('-----------------------------------------------------------------------------------------------')
    print('Input sentence:', xraw[2*seq_index + 1])
    print('Decoded sentence:', decoded_sentence[:-1])

-----------------------------------------------------------------------------------------------
Input sentence: abandon
Decoded sentence: ['HH', 'G', 'F', 'EY', 'ER', 'EH', 'DH', 'D', 'CH', 'B', 'AY', 'AW', 'AO', 'AH', 'AE', 'AA', '\\s', 'R']
-----------------------------------------------------------------------------------------------
Input sentence: abandoning
Decoded sentence: ['HH', 'G', 'F', 'EY', 'ER', 'EH', 'DH', 'D', 'CH', 'B', 'AY', 'AW', 'AO', 'AH', 'AE', 'AA', '\\s', 'R']
-----------------------------------------------------------------------------------------------
Input sentence: abash
Decoded sentence: ['HH', 'G', 'F', 'EY', 'ER', 'EH', 'DH', 'D', 'CH', 'B', 'AY', 'AW', 'AO', 'AH', 'AE', 'AA', '\\s', 'R']
-----------------------------------------------------------------------------------------------
Input sentence: abate
Decoded sentence: ['HH', 'G', 'F', 'EY', 'ER', 'EH', 'DH', 'D', 'CH', 'B', 'AY', 'AW', 'AO', 'AH', 'AE', 'AA', '\\s', 'R']
-----------------------------

-----------------------------------------------------------------------------------------------
Input sentence: abscond
Decoded sentence: ['CH', 'B', 'AY', 'AW', 'AO', 'AH', 'AE', 'AA', '\\s', 'Z', 'Y', 'W', 'V', 'UW', 'UH', 'TH', 'T', 'SH']
-----------------------------------------------------------------------------------------------
Input sentence: absconding
Decoded sentence: ['HH', 'G', 'F', 'EY', 'ER', 'EH', 'DH', 'D', 'CH', 'B', 'AY', 'AW', 'AO', 'AH', 'AE', 'AA', '\\s', 'R']
-----------------------------------------------------------------------------------------------
Input sentence: absent
Decoded sentence: ['HH', 'G', 'F', 'EY', 'ER', 'EH', 'DH', 'D', 'CH', 'B', 'AY', 'AW', 'AO', 'AH', 'AE', 'AA', '\\s', 'R']
-----------------------------------------------------------------------------------------------
Input sentence: absolutely
Decoded sentence: ['HH', 'G', 'F', 'EY', 'ER', 'EH', 'DH', 'D', 'CH', 'B', 'AY', 'AW', 'AO', 'AH', 'AE', 'AA', '\\s', 'R']
------------------------

-----------------------------------------------------------------------------------------------
Input sentence: accomplishing
Decoded sentence: ['HH', 'G', 'F', 'EY', 'ER', 'EH', 'DH', 'D', 'CH', 'B', 'AY', 'AW', 'AO', 'AH', 'AE', 'AA', '\\s', 'R']
-----------------------------------------------------------------------------------------------
Input sentence: accorded
Decoded sentence: ['CH', 'B', 'AY', 'AW', 'AO', 'AH', 'AE', 'AA', '\\s', 'Z', 'Y', 'W', 'V', 'UW', 'UH', 'TH', 'T', 'SH']
-----------------------------------------------------------------------------------------------
Input sentence: accordingly
Decoded sentence: ['CH', 'B', 'AY', 'AW', 'AO', 'AH', 'AE', 'AA', '\\s', 'Z', 'Y', 'W', 'V', 'UW', 'UH', 'TH', 'T', 'SH']
-----------------------------------------------------------------------------------------------
Input sentence: accost
Decoded sentence: ['CH', 'B', 'AY', 'AW', 'AO', 'AH', 'AE', 'AA', '\\s', 'Z', 'Y', 'W', 'V', 'UW', 'UH', 'TH', 'T', 'SH']
---------------------